<a href="https://colab.research.google.com/github/DimaFrank/Association_Rule_Learning/blob/main/OnlineRetail.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install opendatasets

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
!pip install -q kaggle

In [ ]:
from google.colab import files
files.upload()

Saving kaggle.json to kaggle.json


{'kaggle.json': b'{"username":"dimafrank","key":"e7786e191009a5f2647b768669ed5b0c"}'}

In [ ]:
!mkdir -p ~/.kaggle
!cp kaggle.json ~/.kaggle/

In [ ]:
import opendatasets as od

In [ ]:
od.download(
    "https://www.kaggle.com/datasets/vijayuv/onlineretail")

100%|██████████| 7.20M/7.20M [00:00<00:00, 54.5MB/s]

In [ ]:
!pip install pyspark

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 281.4 MB 46 kB/s 
     |████████████████████████████████| 199 kB 55.4 MB/s 
  Created wheel for pyspark: filename=pyspark-3.3.1-py2.py3-none-any.whl size=281845512 sha256=92548fe6d23f4836f4a5fcbb7cae1233aa7b3c35e69334fd5e7dca22d4212635
  Stored in directory: /root/.cache/pip/wheels/43/dc/11/ec201cd671da62fa9c5cc77078235e40722170ceba231d7598
Successfully built pyspark


In [ ]:
import pyspark as spark
from google.colab import drive
from pyspark.sql import SparkSession
from pyspark.sql.types import IntegerType
from pyspark.sql.functions import array, col, concat_ws, udf, array_remove, size

In [ ]:
drive.mount('/content/drive')
!wget --quiet --show-progress https://drive.google.com/drive/u/0/folders/1zKTXVP8ncsRRXgtFxA6MvL-lWirAoc6a

Mounted at /content/drive


In [ ]:
SparkSession = SparkSession.builder.appName("Colab").getOrCreate()

In [71]:
df = SparkSession.read.csv('./onlineretail/OnlineRetail.csv',header='true')

In [72]:
df.show(10)

+---------+---------+--------------------+--------+--------------+---------+----------+--------------+
|InvoiceNo|StockCode|         Description|Quantity|   InvoiceDate|UnitPrice|CustomerID|       Country|
+---------+---------+--------------------+--------+--------------+---------+----------+--------------+
|   536365|   85123A|WHITE HANGING HEA...|       6|12/1/2010 8:26|     2.55|     17850|United Kingdom|
|   536365|    71053| WHITE METAL LANTERN|       6|12/1/2010 8:26|     3.39|     17850|United Kingdom|
|   536365|   84406B|CREAM CUPID HEART...|       8|12/1/2010 8:26|     2.75|     17850|United Kingdom|
|   536365|   84029G|KNITTED UNION FLA...|       6|12/1/2010 8:26|     3.39|     17850|United Kingdom|
|   536365|   84029E|RED WOOLLY HOTTIE...|       6|12/1/2010 8:26|     3.39|     17850|United Kingdom|
|   536365|    22752|SET 7 BABUSHKA NE...|       2|12/1/2010 8:26|     7.65|     17850|United Kingdom|
|   536365|    21730|GLASS STAR FROSTE...|       6|12/1/2010 8:26|     4.

In [ ]:
df.printSchema()

root
 |-- InvoiceNo: string (nullable = true)
 |-- StockCode: string (nullable = true)
 |-- Description: string (nullable = true)
 |-- Quantity: string (nullable = true)
 |-- InvoiceDate: string (nullable = true)
 |-- UnitPrice: string (nullable = true)
 |-- CustomerID: string (nullable = true)
 |-- Country: string (nullable = true)



In [149]:
class UserSessionBuilder:
 
    def __init__(self, datafile, userid, transactions, items, support=0.4, confidence=0.8):
        # datafile      --> Expects to get PySpark DataFrame
        # userid        --> Name of a column that holds UserID.
        # items         --> Name of a column that holds Items in a given dataset
        # transactions  --> Name of a column that holds Transaction ID in a given dataset
        # support       --> Default Treshold is 0.2
        # confidence    --> Required confidence level (Default is 0.8)

        self.datafile=datafile
        self.userid=userid
        self.transactions=transactions
        self.items=items
        self.support=support
        self.confidence=confidence


    def create_support(self):
        self.datafile.createOrReplaceTempView("temp")
        query = '''
                WITH SOURCE as (
                                SELECT d.*, REPLACE({ITEMS}, ' ', '_') as ITEMS                                                  
                                FROM {data} as d
                                WHERE {ITEMS} <> 'null'
                                ),

                    TOTAL as ( select count(distinct ITEMS) from SOURCE ),          

                    SUPPORT_ITEM_SET1 (
                          SELECT *, CNT/ALL_CNT as SUPPORT 
                          FROM(
                              SELECT ITEMS, count(*) as CNT, (select * from TOTAL) as ALL_CNT 
                              FROM SOURCE
                              GROUP BY ITEMS
                              ) as tab1
                          WHERE (CNT/ALL_CNT) > {min_support}
                          ORDER BY SUPPORT DESC
                                      )
                                    
                    SELECT *
                    FROM SOURCE
                    WHERE ITEMS in (SELECT ITEMS FROM SUPPORT_ITEM_SET1)

              '''.format(data='temp', ITEMS=self.items, min_support=self.support)

        SparkSession.sql(query).createOrReplaceTempView("support")
        return SparkSession.sql(query)


    def create_basket(self):
        global item_mapping 
        basket = self.create_support()
        basket = basket.groupBy(['{}'.format(self.userid),'{}'.format(self.transactions) ]).pivot('{}'.format(self.items)).count().fillna(value=0).sort(col("CustomerID").desc())
        new_col_names = [self.userid,self.transactions] + ['ITEM{i}'.format(i=i) for i in range(len(basket.schema.names)-1) if i>=1]  
        item_mapping = {new_col_names[i]: basket.schema.names[i] for i in range(len(basket.schema.names)) if i>=2}
        basket = basket.toDF(*new_col_names)
        basket = basket.select("{}".format(self.userid),"{}".format(self.transactions), array(*new_col_names[2:]).alias("Basket")) 
        basket = basket.withColumn(
                "BasketBinaryVector",
                   spark.sql.functions.expr("""transform(
                              Basket, 
                              (x, i) -> CASE WHEN x > 0 THEN 1
                                            ELSE x
                                        END
                          )
                      """)
                  )
        
        basket = basket.withColumn(
                "BasketVector",
                   spark.sql.functions.expr("""transform(
                              Basket, 
                              (x, i) -> CASE WHEN x > 0 THEN concat("ITEM",i+1)
                                            ELSE ""
                                        END
                          )
                      """)
                  )
        
        basket = basket.select(self.userid, self.transactions, "Basket", "BasketBinaryVector" ,array_remove('BasketVector',""))
        basket = basket.withColumn("size", size(col("array_remove(BasketVector, )"))).withColumnRenamed("array_remove(BasketVector, )", "BasketClean")
        basket.createOrReplaceTempView("basket")
        return basket


    def create_item_sets(self, display=False):
        global max_item_set
        basket = self.create_basket()
        max_item_set = SparkSession.sql("SELECT MAX(size) FROM basket").collect()[0][0]
        i=1
        while i<=max_item_set:         
          temp = SparkSession.sql('''
                                  SELECT {USER}, BasketClean, count(*) as CNT
                                  FROM basket 
                                  WHERE size == {SIZE} and {USER} != 'null'
                                  GROUP BY {USER}, BasketClean   
                                  ORDER BY CNT DESC   
                                  '''.format(USER=self.userid,SIZE=i))

          temp.createOrReplaceTempView("item_set_{n}".format(n=i))  
          if display:
            temp.show(5)  

          i+=1

      
    def fit(self):
      self.create_item_sets()
      Sx = ['ITEM1', 'ITEM3']
      user = '14096'

      temp = SparkSession.sql('''
                              SELECT *
                              FROM item_set_{set_size}                                             
                              WHERE {USER_ID}=={USER}

                              '''.format(USER_ID=self.userid, USER=user, set_size=len(Sx)))

      temp.createOrReplaceTempView("item_set_{n}_test".format(n=len(Sx)))  

      return temp



    def get_mapping(self):
       self.create_basket()
       return item_mapping



In [68]:
class RuleMining:

   def __init__(self, data, Sx, Sy, confidence=0.8):
      self.data=data
      self.Sx=Sx
      self.Sy=Sy
      self.confidence=confidence
  

In [96]:
UserSessionBuilder(df, 'CustomerID', 'InvoiceNo', 'Description').create_support().show(10)

+---------+---------+--------------------+--------+---------------+---------+----------+--------------+--------------------+
|InvoiceNo|StockCode|         Description|Quantity|    InvoiceDate|UnitPrice|CustomerID|       Country|               ITEMS|
+---------+---------+--------------------+--------+---------------+---------+----------+--------------+--------------------+
|   536365|   85123A|WHITE HANGING HEA...|       6| 12/1/2010 8:26|     2.55|     17850|United Kingdom|WHITE_HANGING_HEA...|
|   536373|   85123A|WHITE HANGING HEA...|       6| 12/1/2010 9:02|     2.55|     17850|United Kingdom|WHITE_HANGING_HEA...|
|   536375|   85123A|WHITE HANGING HEA...|       6| 12/1/2010 9:32|     2.55|     17850|United Kingdom|WHITE_HANGING_HEA...|
|   536386|   85099B|JUMBO BAG RED RET...|     100| 12/1/2010 9:57|     1.65|     16029|United Kingdom|JUMBO_BAG_RED_RET...|
|   536390|   85123A|WHITE HANGING HEA...|      64|12/1/2010 10:19|     2.55|     17511|United Kingdom|WHITE_HANGING_HEA...|


In [133]:
UserSessionBuilder(df, 'CustomerID', 'InvoiceNo', 'Description').create_basket().show(10)

+----------+---------+------------+------------------+--------------+----+
|CustomerID|InvoiceNo|      Basket|BasketBinaryVector|   BasketClean|size|
+----------+---------+------------+------------------+--------------+----+
|     18283|   578262|[1, 0, 0, 0]|      [1, 0, 0, 0]|       [ITEM1]|   1|
|     18283|   556731|[1, 0, 0, 1]|      [1, 0, 0, 1]|[ITEM1, ITEM4]|   2|
|     18283|   557956|[2, 0, 0, 1]|      [1, 0, 0, 1]|[ITEM1, ITEM4]|   2|
|     18283|   565579|[1, 0, 0, 0]|      [1, 0, 0, 0]|       [ITEM1]|   1|
|     18283|   545079|[0, 0, 0, 1]|      [0, 0, 0, 1]|       [ITEM4]|   1|
|     18283|   575668|[1, 0, 0, 0]|      [1, 0, 0, 0]|       [ITEM1]|   1|
|     18283|   550957|[1, 0, 0, 0]|      [1, 0, 0, 0]|       [ITEM1]|   1|
|     18283|   579673|[1, 0, 0, 0]|      [1, 0, 0, 0]|       [ITEM1]|   1|
|     18283|   540350|[1, 0, 0, 0]|      [1, 0, 0, 0]|       [ITEM1]|   1|
|     18283|   560025|[1, 0, 0, 1]|      [1, 0, 0, 1]|[ITEM1, ITEM4]|   2|
+----------+---------+---

In [151]:
UserSessionBuilder(df, 'CustomerID', 'InvoiceNo', 'Description').create_item_sets(display=True)

+----------+-----------+---+
|CustomerID|BasketClean|CNT|
+----------+-----------+---+
|     14911|    [ITEM3]| 46|
|     14911|    [ITEM4]| 25|
|     15311|    [ITEM1]| 20|
|     12471|    [ITEM3]| 20|
|     15465|    [ITEM3]| 20|
+----------+-----------+---+
only showing top 5 rows

+----------+--------------+---+
|CustomerID|   BasketClean|CNT|
+----------+--------------+---+
|     16458|[ITEM1, ITEM4]| 10|
|     13777|[ITEM1, ITEM4]|  9|
|     13263|[ITEM3, ITEM4]|  7|
|     17675|[ITEM2, ITEM4]|  7|
|     13097|[ITEM2, ITEM3]|  6|
+----------+--------------+---+
only showing top 5 rows

+----------+--------------------+---+
|CustomerID|         BasketClean|CNT|
+----------+--------------------+---+
|     13081|[ITEM2, ITEM3, IT...|  5|
|     17675|[ITEM1, ITEM2, IT...|  3|
|     17652|[ITEM1, ITEM2, IT...|  3|
|     13334|[ITEM1, ITEM2, IT...|  3|
|     14096|[ITEM1, ITEM3, IT...|  3|
+----------+--------------------+---+
only showing top 5 rows

+----------+--------------------+-

In [152]:
UserSessionBuilder(df, 'CustomerID', 'InvoiceNo', 'Description').fit().show(5)

+----------+--------------+---+
|CustomerID|   BasketClean|CNT|
+----------+--------------+---+
|     14096|[ITEM3, ITEM4]|  2|
|     14096|[ITEM1, ITEM3]|  1|
+----------+--------------+---+



In [153]:
UserSessionBuilder(df, 'CustomerID', 'InvoiceNo', 'Description').get_mapping()

{'ITEM1': 'JUMBO BAG RED RETROSPOT',
 'ITEM2': 'PARTY BUNTING',
 'ITEM3': 'REGENCY CAKESTAND 3 TIER',
 'ITEM4': 'WHITE HANGING HEART T-LIGHT HOLDER'}

In [141]:
241/871

0.2766934557979334